# Tagging our images: church or not church?

Now that we have our overhead images, we want to inspect them and tag them based on whether they're a purpose-built church building or not. The __[PigeonXT library](https://github.com/dennisbakhuis/pigeonXT)__ library provides a straightforward way to do it in Jupyter Lab with minimal tweaking.

In [1]:
import pandas as pd
import pigeonXT as pixt
from IPython.display import display, Image
import os

Load in our images:

In [2]:
candidates = [os.path.join('images',file) for file in os.listdir('images/') if file.endswith('.png')]
candidates[0:3]

['images\\abbeyfield_community_church.png',
 'images\\all_saints_eight_ash_green.png',
 'images\\ardleigh_methodist_church.png']

Now we'll see if there's already an annotations file present. If there is, we can skip this step; if not, we can tag all the images. The tagging interface looks like this:
![Image of a church with several buttons above it to select a tag: chuch, not church, unclear, prev and next.](writeup_files/image_tagging_example.PNG)

In [3]:
if os.path.exists('annotations.csv'):
    annotations = pd.read_csv('annotations.csv')
else:
    annotations = pixt.annotate(candidates,
                                options=['church','not church','unclear'],
                                display_fn=lambda candidate: display(Image(candidate))
                               )
    annotations.to_csv('annotations.csv', index=False)

We end up with a dataframe with a handful of columns – here 21 buildings are clearly churches, and 28 are unclear. This could be resolved by a quick Google or by using Streetview imagery.

In [4]:
annotations.head()

,example,changed,label,filename
0,images\abbeyfield_community_church.png,True,unclear,abbeyfield_community_church
1,images\all_saints_eight_ash_green.png,True,church,all_saints_eight_ash_green
2,images\ardleigh_methodist_church.png,True,unclear,ardleigh_methodist_church
3,images\castle_methodist_church.png,True,unclear,castle_methodist_church
4,images\catholic_church_of_saint_joseph.png,True,unclear,catholic_church_of_saint_joseph


In [5]:
annotations['label'].value_counts()

unclear    28
church     21
Name: label, dtype: int64

Now we create a filename column (if not already present) to use as the merge key with the main places of worship dataframe:

In [6]:
annotations['filename'] = annotations['example'].apply(lambda x: x.split('\\')[1].split('.')[0])
annotations.head()

,example,changed,label,filename
0,images\abbeyfield_community_church.png,True,unclear,abbeyfield_community_church
1,images\all_saints_eight_ash_green.png,True,church,all_saints_eight_ash_green
2,images\ardleigh_methodist_church.png,True,unclear,ardleigh_methodist_church
3,images\castle_methodist_church.png,True,unclear,castle_methodist_church
4,images\catholic_church_of_saint_joseph.png,True,unclear,catholic_church_of_saint_joseph


In [7]:
places_of_worship = pd.read_csv('places_of_worship.csv')
places_of_worship.head()

,id,tags.name,tags.denomination,tags.building,lat,lon,coords_imputed,filename
0,144164756,"St Margaret and St Catherine, Aldham",anglican,NaN,51.898162,0.785737,True,st_margaret_and_st_catherine_aldham
1,145482410,"St Michael and All Angels, Copford",anglican,NaN,51.869317,0.809385,True,st_michael_and_all_angels_copford
2,35139156,St. Albright's Church,anglican,NaN,51.883674,0.817888,True,st_albright_s_church
3,146823677,"All Saints, Eight Ash Green",anglican,NaN,51.896594,0.820112,True,all_saints_eight_ash_green
4,913980214,Eight Ash Green Methodist Church,NaN,NaN,51.895918,0.822322,False,eight_ash_green_methodist_church


In [8]:
places_of_worship = places_of_worship.merge(right=annotations[['label','filename']], on='filename')

Finally, save the results:

In [9]:
places_of_worship.to_csv('labelled_results.csv')

The unclear results can be followed up and corrected manually.

***